In [1]:
!pip install kaggle -q

In [2]:
# importing the dependencies
import os
import json

import zipfile
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
### Data Collection
kaggle_config = json.load(open("kaggle.json"))

In [4]:
kaggle_config.keys()

dict_keys(['username', 'key'])

In [5]:
### Set the environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_config['username']
os.environ['KAGGLE_KEY'] = kaggle_config['key']

In [6]:
### Loading the dataset using kaggle API
!kaggle datasets download notshrirang/spotify-million-song-dataset

Dataset URL: https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset
License(s): CC0-1.0


In [7]:
!ls

Music Recommended System.ipynb
README.md
kaggle.json
spotify-million-song-dataset.zip


In [8]:
with zipfile.ZipFile('spotify-million-song-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()